# Creating Crosswalk with DLA Locode, NTD ID and ITP ID

In [1]:
import pandas as pd
from siuba import *
import numpy as np
from calitp import *

In [2]:
GCS_FILE_PATH = 'gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/'

pd.set_option('display.max_columns', None)

In [3]:
dla = pd.read_excel(f"{GCS_FILE_PATH}locodes_updated7122021.xlsx")

/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [4]:
dla = to_snakecase(dla)
dla = dla.rename(columns = {'agency_name':'name', 'agency_locode':'ct_code'})

In [5]:
dla.sample(1)

,ct_code,name,district,county_name,rtpa_name,mpo_name,mpo_locode_fads,active_e76s______7_12_2021_
807,5948,Inyo County,9,Inyo County,Inyo County Transportation Commission,NON-MPO,NON-MPO,Yes


In [6]:
dla = dla>>select(_.ct_code, _.name, _.district)

### Reading in [Place Names](https://dot.ca.gov/-/media/dot-media/programs/research-innovation-system-information/documents/place-names/2020-place-names-in-california-a11y.pdf)

In [7]:
pl_city = pd.read_excel(f"{GCS_FILE_PATH}2020-place-names-locode.xlsx", sheet_name=0)
pl_county = pd.read_excel(f"{GCS_FILE_PATH}2020-place-names-locode.xlsx", sheet_name=1)

In [8]:
pl_city = to_snakecase(pl_city)
pl_county = to_snakecase(pl_county)

In [9]:
pl_city = pl_city.drop(columns={'unnamed:_1','unnamed:_3','unnamed:_4','unnamed:_6','unnamed:_7', 'date_of_incorporation', 'urban_area_no_'})
pl_county = pl_county.drop(columns={'unnamed:_1','unnamed:_3','unnamed:_4','unnamed:_6','unnamed:_9'})

In [10]:
pl_city = pl_city.rename(columns={'ct_city_code':'ct_code', 'city_name_abbr_':'abbr', 'dist_':'district', 'co_':'county'})
pl_county = pl_county.rename(columns={'ct_co__code':'ct_code', 'county_name':'name', 'co__name_abbr_':'abbr'})

In [11]:
pl_city['organization_type']= ('City/Town')
pl_county['organization_type']= ('County')

In [12]:
pl_county['name'] = pl_county['name'].astype(str) + ' County' 

In [13]:
pl_county.ct_code.fillna(pl_county['unnamed:_8'], inplace=True)
pl_county = pl_county.drop(columns = {'unnamed:_8'})

In [14]:
plnames = pd.concat([pl_city, pl_county])

In [15]:
plnames

,name,abbr,district,county,ct_code,organization_type,co__fips
0,Adelanto,Adel,8,SBd,5408.0,City/Town,NaN
1,Agoura Hills,AgrH,7,LA,5435.0,City/Town,NaN
2,Alameda,Ala,4,Ala,5014.0,City/Town,NaN
3,Albany,Alb,4,Ala,5178.0,City/Town,NaN
4,Alhambra,Alh,7,LA,5130.0,City/Town,NaN
...,...,...,...,...,...,...,...
53,Tulare County,Tul,6,NaN,5946.0,County,107.0
54,Tuolumne County,Tuo,10,NaN,5932.0,County,109.0
55,Ventura County,Ven,7,NaN,5952.0,County,111.0
56,Yolo County,Yol,3,NaN,5922.0,County,113.0


In [16]:
plnames.county.fillna(plnames.abbr, inplace=True)

In [17]:
fips_map = dict(zip(pl_county['abbr'], 
                          pl_county['co__fips']))

In [18]:
plnames['co__fips'] = plnames['county'].map(fips_map)

In [19]:
type_map = dict(zip(plnames['ct_code'], 
                          plnames['organization_type']))

In [20]:
dla['organization_type'] = dla['ct_code'].map(type_map)

In [21]:
mask1 = (dla['name'].str.contains('Council of Governments'))
mask2 = (dla['name'].str.contains('Tribe')) 
mask3 = (dla['name'].str.contains('University'))
mask4 = (dla['name'].str.contains('RTPA'))
mask5 = (dla['name'].str.contains('JPA'))

dla.loc[dla['organization_type'].isnull() & mask1, 'organization_type'] = 'Council of Governments'
dla.loc[dla['organization_type'].isnull() & mask2, 'organization_type'] = 'Tribe'
dla.loc[dla['organization_type'].isnull() & mask3, 'organization_type'] = 'University - Public'
dla.loc[dla['organization_type'].isnull() & mask4, 'organization_type'] = 'MPO/RTPA'
dla.loc[dla['organization_type'].isnull() & mask5, 'organization_type'] = 'Joint Powers Agency'

In [22]:
#dla.to_csv('dla_placenames.csv')

### READ ITP

In [26]:
import calitp
from calitp.tables import tbl

In [30]:
itp = pd.read_csv(f"{GCS_FILE_PATH}organizations-All Organizations_0427.csv")

In [31]:
itp = to_snakecase(itp)

In [32]:
itp = itp>>select(_.name, _.ntp_id, _.itp_id, _.organization_type)

In [33]:
itp = itp.rename(columns = {"ntp_id":"ntd_id"})

In [34]:
itp

,name,ntd_id,itp_id,organization_type
0,A-Paratransit,NaN,NaN,NaN
1,ABC Shuttle,NaN,NaN,Company
2,Able Inc.,NaN,NaN,Non-Profit Organization
3,Abrazar Inc.,NaN,NaN,Non-Profit Organization
4,Access Services,90157,1.0,NaN
...,...,...,...,...
836,Yosemite Area Regional Transportation System,9R02-91070,374.0,Independent Agency
837,Yuba-Sutter Transit Authority,90061,376.0,Independent Agency
838,Yuma County Intergovernmental Public Transport...,90233,386.0,Independent Agency
839,Yurok Tribe,99262,377.0,Tribe


## Merging

In [35]:
ids = pd.merge(itp, dla, on='name', how="outer", indicator=True)

In [36]:
ids._merge.value_counts()

right_only    923
left_only     723
both          118
Name: _merge, dtype: int64

* right only is DLA
* left only is ITP/NTD ID

In [37]:
ids>>count(_.name)>>arrange(-_.n)>>filter(_.n>1)

,name,n
189,Caltrans,13
1106,North East Trees,2
1661,Urban Corps of San Diego,2


In [38]:
ids>>filter(_.name=='North East Trees')

,name,ntd_id,itp_id,organization_type_x,ct_code,district,organization_type_y,_merge
1379,North East Trees,NaN,NaN,NaN,6118.0,7.0,NaN,right_only
1380,North East Trees,NaN,NaN,NaN,6434.0,7.0,NaN,right_only


In [39]:
ids>>filter(_.name=='Urban Corps of San Diego')

,name,ntd_id,itp_id,organization_type_x,ct_code,district,organization_type_y,_merge
1629,Urban Corps of San Diego,NaN,NaN,NaN,6490.0,11.0,NaN,right_only
1630,Urban Corps of San Diego,NaN,NaN,NaN,6233.0,11.0,NaN,right_only


In [40]:
#duplicates expected
ids>>filter(_.name=='Caltrans')

,name,ntd_id,itp_id,organization_type_x,ct_code,district,organization_type_y,_merge
858,Caltrans,NaN,NaN,NaN,6201.0,1.0,NaN,right_only
859,Caltrans,NaN,NaN,NaN,6202.0,2.0,NaN,right_only
860,Caltrans,NaN,NaN,NaN,6203.0,3.0,NaN,right_only
861,Caltrans,NaN,NaN,NaN,6204.0,4.0,NaN,right_only
862,Caltrans,NaN,NaN,NaN,6205.0,5.0,NaN,right_only
863,Caltrans,NaN,NaN,NaN,6206.0,6.0,NaN,right_only
864,Caltrans,NaN,NaN,NaN,6207.0,7.0,NaN,right_only
865,Caltrans,NaN,NaN,NaN,6208.0,8.0,NaN,right_only
866,Caltrans,NaN,NaN,NaN,6209.0,9.0,NaN,right_only
867,Caltrans,NaN,NaN,NaN,6210.0,10.0,NaN,right_only


In [41]:
#interesting that DLA Locode has some duplciates... will investigate
#caltrans duplicates expected

#### Adding Organization Types

In [42]:
ids.organization_type_y.fillna(ids['organization_type_x'], inplace=True)

In [43]:
ids = ids.rename(columns={'organization_type_y':'organization_type'})
ids.drop(columns=('organization_type_x'), inplace=True)

In [44]:
ids

,name,ntd_id,itp_id,ct_code,district,organization_type,_merge
0,A-Paratransit,NaN,NaN,NaN,NaN,NaN,left_only
1,ABC Shuttle,NaN,NaN,NaN,NaN,Company,left_only
2,Able Inc.,NaN,NaN,NaN,NaN,Non-Profit Organization,left_only
3,Abrazar Inc.,NaN,NaN,NaN,NaN,Non-Profit Organization,left_only
4,Access Services,90157,1.0,6312.0,7.0,NaN,both
...,...,...,...,...,...,...,...
1759,Leave Blank,NaN,NaN,5465.0,53.0,NaN,right_only
1760,U.S. Fish and Wildlife Service,NaN,NaN,6250.0,53.0,NaN,right_only
1761,Tidewater Southern Railway Company,NaN,NaN,6031.0,53.0,NaN,right_only
1762,Tri-Counties Regional Park Group,NaN,NaN,6176.0,53.0,NaN,right_only


In [45]:
mask1 = (ids['name'].str.contains('Council of Governments'))
mask2 = (ids['name'].str.contains('Association of Governments')) 
mask4 = (ids['name'].str.contains('Transportation Commission'))
mask5 = (ids['name'].str.contains('Joint Powers Agency'))
mask6 = (ids['name'].str.contains('Transportation Agency'))
mask8 = (ids['name'].str.contains('Port Hueneme'))
mask9 = (ids['name'].str.contains('Access Services'))
mask10 = (ids['name'].str.contains('County Transportation Authority'))


ids.loc[ids['organization_type'].isnull() & mask1, 'organization_type'] = 'Council of Governments'
ids.loc[ids['organization_type'].isnull() & mask2, 'organization_type'] = 'Council of Governments'
ids.loc[ids['organization_type'].isnull() & mask4, 'organization_type'] = 'Transportation Commission'
ids.loc[ids['organization_type'].isnull() & mask5, 'organization_type'] = 'Joint Powers Agency'
ids.loc[ids['organization_type'].isnull() & mask6, 'organization_type'] = 'MPO/RTPA'
ids.loc[ids['organization_type'].isnull() & mask8, 'organization_type'] = 'City/Town'
ids.loc[ids['organization_type'].isnull() & mask9, 'organization_type'] = 'Independent Agency'
ids.loc[ids['organization_type'].isnull() & mask10, 'organization_type'] = 'Transportation Commission'

In [46]:
mask11 = (ids['name'].str.contains('Regional'))

In [47]:
ids.organization_type.value_counts()

City/Town                    755
Company                      159
Non-Profit Organization       94
Independent Agency            75
County                        58
Transportation Commission     39
University - Public           34
Council of Governments        22
Joint Powers Agency            7
Tribe                          7
MPO/RTPA                       6
University - Private           5
Federal Government             2
Community College              2
Name: organization_type, dtype: int64

In [48]:
(
    ids
    >> filter(_.organization_type.isnull())
    >> filter(_.name.str.contains("Governments"))
)

,name,ntd_id,itp_id,ct_code,district,organization_type,_merge
384,Council of San Benito County Governments,NaN,NaN,6060.0,5.0,NaN,both
1239,Association Of Monterey Bay Area Governments,NaN,NaN,6091.0,5.0,NaN,right_only
1272,Council of Fresno County Governments,NaN,NaN,6086.0,6.0,NaN,right_only
1475,Western Riverside Council Of Governments,NaN,NaN,6107.0,8.0,NaN,right_only
1476,San Bernardino Associated Governments,NaN,NaN,6053.0,8.0,NaN,right_only


#### Subset for Orgs that matched

In [49]:
both = (ids>>filter(_._merge=='both'))

In [50]:
both = both.drop(columns=['_merge'])

In [51]:
both

,name,ntd_id,itp_id,ct_code,district,organization_type
4,Access Services,90157,1.0,6312.0,7.0,Independent Agency
13,Alpine County,9R02-91116,9.0,5931.0,10.0,County
15,Amador County,NaN,NaN,5926.0,10.0,County
16,Amador County Transportation Commission,NaN,NaN,6127.0,10.0,Transportation Commission
28,Antelope Valley Transit Authority,90121,16.0,6166.0,7.0,Independent Agency
...,...,...,...,...,...,...
810,Ventura County Transportation Commission,90164,380.0,6155.0,7.0,Transportation Commission
815,Victor Valley Transit Authority,90148,360.0,6261.0,8.0,Independent Agency
828,Western Contra Costa Transit Authority,90159,368.0,6226.0,4.0,Independent Agency
835,Yolo County Transportation District,90090,372.0,6195.0,3.0,Independent Agency


In [52]:
both.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118 entries, 4 to 837
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   name               118 non-null    object 
 1   ntd_id             57 non-null     object 
 2   itp_id             66 non-null     float64
 3   ct_code            118 non-null    float64
 4   district           118 non-null    float64
 5   organization_type  112 non-null    object 
dtypes: float64(3), object(3)
memory usage: 6.5+ KB


In [53]:
both['organization_type'].value_counts()

Independent Agency           42
County                       36
Transportation Commission    15
Council of Governments       12
Joint Powers Agency           3
MPO/RTPA                      2
City/Town                     1
Community College             1
Name: organization_type, dtype: int64

In [54]:
nomatch = ids>>filter(_._merge!='both')

In [55]:
nomatch.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1646 entries, 0 to 1763
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   name               1646 non-null   object  
 1   ntd_id             142 non-null    object  
 2   itp_id             309 non-null    float64 
 3   ct_code            923 non-null    float64 
 4   district           923 non-null    float64 
 5   organization_type  1153 non-null   object  
 6   _merge             1646 non-null   category
dtypes: category(1), float64(3), object(3)
memory usage: 91.8+ KB


In [56]:
nomatch>>count(_._merge)

,_merge,n
0,left_only,723
1,right_only,923
2,both,0


In [57]:
(nomatch.query('itp_id.notnull() and ntd_id.notnull()')).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 135 entries, 10 to 839
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   name               135 non-null    object  
 1   ntd_id             135 non-null    object  
 2   itp_id             135 non-null    float64 
 3   ct_code            0 non-null      float64 
 4   district           0 non-null      float64 
 5   organization_type  128 non-null    object  
 6   _merge             135 non-null    category
dtypes: category(1), float64(3), object(3)
memory usage: 7.6+ KB


In [58]:
nomatch.query('itp_id.isnull() or ntd_id.isnull() or ct_code.isnull()')

,name,ntd_id,itp_id,ct_code,district,organization_type,_merge
0,A-Paratransit,NaN,NaN,NaN,NaN,NaN,left_only
1,ABC Shuttle,NaN,NaN,NaN,NaN,Company,left_only
2,Able Inc.,NaN,NaN,NaN,NaN,Non-Profit Organization,left_only
3,Abrazar Inc.,NaN,NaN,NaN,NaN,Non-Profit Organization,left_only
5,ACE Parking,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...
1759,Leave Blank,NaN,NaN,5465.0,53.0,NaN,right_only
1760,U.S. Fish and Wildlife Service,NaN,NaN,6250.0,53.0,NaN,right_only
1761,Tidewater Southern Railway Company,NaN,NaN,6031.0,53.0,NaN,right_only
1762,Tri-Counties Regional Park Group,NaN,NaN,6176.0,53.0,NaN,right_only


In [59]:
no_ids = nomatch>>filter(_.itp_id.isnull())>>filter(_.ntd_id.isnull())

In [60]:
no_ids>>filter(_.ct_code.isnull())

,name,ntd_id,itp_id,ct_code,district,organization_type,_merge
0,A-Paratransit,NaN,NaN,NaN,NaN,NaN,left_only
1,ABC Shuttle,NaN,NaN,NaN,NaN,Company,left_only
2,Able Inc.,NaN,NaN,NaN,NaN,Non-Profit Organization,left_only
3,Abrazar Inc.,NaN,NaN,NaN,NaN,Non-Profit Organization,left_only
5,ACE Parking,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...
830,Westminster on Wheels Senior Transportation Pr...,NaN,NaN,NaN,NaN,NaN,left_only
831,Willits Seniors Inc.,NaN,NaN,NaN,NaN,Non-Profit Organization,left_only
833,Worldpay,NaN,NaN,NaN,NaN,Company,left_only
834,Yellow Cab CA,NaN,NaN,NaN,NaN,Company,left_only


In [61]:
both.query('itp_id.isnull() or ntd_id.isnull()')

,name,ntd_id,itp_id,ct_code,district,organization_type
15,Amador County,NaN,NaN,5926.0,10.0,County
16,Amador County Transportation Commission,NaN,NaN,6127.0,10.0,Transportation Commission
57,Butte County,NaN,NaN,5912.0,3.0,County
61,Calaveras Council of Governments,NaN,NaN,6128.0,10.0,Council of Governments
373,Colusa County Transportation Commission,NaN,NaN,6129.0,3.0,Transportation Commission
...,...,...,...,...,...,...
771,Transportation Agency for Monterey County,NaN,NaN,6143.0,5.0,MPO/RTPA
777,Trinity County Transportation Commission,NaN,NaN,6153.0,2.0,Transportation Commission
780,Tulare County,NaN,NaN,5946.0,6.0,County
782,Tulare County Association of Governments,NaN,NaN,6094.0,6.0,Joint Powers Agency


In [62]:
both.query('organization_type.isnull()')

,name,ntd_id,itp_id,ct_code,district,organization_type
384,Council of San Benito County Governments,NaN,NaN,6060.0,5.0,NaN
505,Lake County/City Area Planning Council,NaN,NaN,6136.0,1.0,NaN
618,Placer County Transportation Planning Agency,NaN,NaN,6158.0,3.0,NaN
705,Shasta County Regional Transportation Planning...,NaN,NaN,6093.0,2.0,NaN
751,Tahoe Regional Planning Agency,NaN,NaN,6125.0,3.0,NaN
786,Tuolumne County Transportation Council,NaN,NaN,6154.0,10.0,NaN


In [63]:
# Most Agency types that are null are Transportation Commissions, or COGs. 

In [64]:
print(f"Out of {(len(ids>>filter(_.ct_code.notnull())))} organizations in the Official Locode List, {(len(ids>>filter(_.ct_code.notnull())>>filter(_.organization_type.notnull())))} have an Organization Type listed in Airtable")


Out of 1041 organizations in the Official Locode List, 660 have an Organization Type listed in Airtable


In [65]:
ids.organization_type.value_counts()

City/Town                    755
Company                      159
Non-Profit Organization       94
Independent Agency            75
County                        58
Transportation Commission     39
University - Public           34
Council of Governments        22
Joint Powers Agency            7
Tribe                          7
MPO/RTPA                       6
University - Private           5
Federal Government             2
Community College              2
Name: organization_type, dtype: int64

In [66]:
# There is a `Council of Governments` option available 

In [67]:
(both.query('organization_type.isnull()'))

,name,ntd_id,itp_id,ct_code,district,organization_type
384,Council of San Benito County Governments,NaN,NaN,6060.0,5.0,NaN
505,Lake County/City Area Planning Council,NaN,NaN,6136.0,1.0,NaN
618,Placer County Transportation Planning Agency,NaN,NaN,6158.0,3.0,NaN
705,Shasta County Regional Transportation Planning...,NaN,NaN,6093.0,2.0,NaN
751,Tahoe Regional Planning Agency,NaN,NaN,6125.0,3.0,NaN
786,Tuolumne County Transportation Council,NaN,NaN,6154.0,10.0,NaN


In [68]:
both>>filter(_.ntd_id.isnull())

,name,ntd_id,itp_id,ct_code,district,organization_type
15,Amador County,NaN,NaN,5926.0,10.0,County
16,Amador County Transportation Commission,NaN,NaN,6127.0,10.0,Transportation Commission
57,Butte County,NaN,NaN,5912.0,3.0,County
61,Calaveras Council of Governments,NaN,NaN,6128.0,10.0,Council of Governments
373,Colusa County Transportation Commission,NaN,NaN,6129.0,3.0,Transportation Commission
...,...,...,...,...,...,...
771,Transportation Agency for Monterey County,NaN,NaN,6143.0,5.0,MPO/RTPA
777,Trinity County Transportation Commission,NaN,NaN,6153.0,2.0,Transportation Commission
780,Tulare County,NaN,NaN,5946.0,6.0,County
782,Tulare County Association of Governments,NaN,NaN,6094.0,6.0,Joint Powers Agency


In [69]:
both>>filter(_.itp_id.isnull())

,name,ntd_id,itp_id,ct_code,district,organization_type
15,Amador County,NaN,NaN,5926.0,10.0,County
16,Amador County Transportation Commission,NaN,NaN,6127.0,10.0,Transportation Commission
57,Butte County,NaN,NaN,5912.0,3.0,County
61,Calaveras Council of Governments,NaN,NaN,6128.0,10.0,Council of Governments
373,Colusa County Transportation Commission,NaN,NaN,6129.0,3.0,Transportation Commission
384,Council of San Benito County Governments,NaN,NaN,6060.0,5.0,NaN
410,El Dorado County Transportation Commission,NaN,NaN,6157.0,3.0,Transportation Commission
436,Fresno Council of Governments,NaN,NaN,6504.0,6.0,Council of Governments
437,Fresno County,NaN,NaN,5942.0,6.0,County
449,Glenn County Transportation Commission,NaN,NaN,6132.0,3.0,Transportation Commission


In [70]:
# many in both have blank NTD and ITP ids but are still identified in Airtable

In [71]:
print(f"Out of the {(len(both))} organizations that matched, {(len(both.query('itp_id.notnull() and ntd_id.notnull() and organization_type.notnull()')))} have all id fields filled (ITP/NTD/Locode)")


Out of the 118 organizations that matched, 54 have all id fields filled (ITP/NTD/Locode)


In [72]:
#the 54 with all fields filled out
(both.query('itp_id.notnull() and ntd_id.notnull() and organization_type.notnull()'))

,name,ntd_id,itp_id,ct_code,district,organization_type
4,Access Services,90157,1.0,6312.0,7.0,Independent Agency
13,Alpine County,9R02-91116,9.0,5931.0,10.0,County
28,Antelope Valley Transit Authority,90121,16.0,6166.0,7.0,Independent Agency
58,Butte County Association of Governments,90208,47.0,6092.0,3.0,Joint Powers Agency
62,Calaveras County,9R02-91063,50.0,5930.0,10.0,County
409,El Dorado County Transit Authority,90229,101.0,6225.0,3.0,Independent Agency
438,Fresno County Rural Transit Agency,9R02-91007,117.0,6194.0,6.0,Independent Agency
448,Glenn County,9R02-91088,122.0,5911.0,3.0,County
454,Golden Empire Transit District,90004,126.0,6013.0,6.0,Independent Agency
472,Humboldt Transit Authority,9R02-91036,135.0,6162.0,1.0,Independent Agency


In [73]:
print(f"If we just look at completed Locodes (no ITP/NTD), we now have organization type info for {(len(both.query('ct_code.notnull() and organization_type.notnull()')))} organizations")


If we just look at completed Locodes (no ITP/NTD), we now have organization type info for 112 organizations


In [74]:
#both.to_parquet(f"{GCS_FILE_PATH}ITP_NTD_Locode_crosswalk.parquet")

#this just has the Agency Locode, NTD ID, ITP ID and Agency Locode

## Full Merged Data

In [75]:
ids

,name,ntd_id,itp_id,ct_code,district,organization_type,_merge
0,A-Paratransit,NaN,NaN,NaN,NaN,NaN,left_only
1,ABC Shuttle,NaN,NaN,NaN,NaN,Company,left_only
2,Able Inc.,NaN,NaN,NaN,NaN,Non-Profit Organization,left_only
3,Abrazar Inc.,NaN,NaN,NaN,NaN,Non-Profit Organization,left_only
4,Access Services,90157,1.0,6312.0,7.0,Independent Agency,both
...,...,...,...,...,...,...,...
1759,Leave Blank,NaN,NaN,5465.0,53.0,NaN,right_only
1760,U.S. Fish and Wildlife Service,NaN,NaN,6250.0,53.0,NaN,right_only
1761,Tidewater Southern Railway Company,NaN,NaN,6031.0,53.0,NaN,right_only
1762,Tri-Counties Regional Park Group,NaN,NaN,6176.0,53.0,NaN,right_only


In [76]:
print(f"Althought we still have {len(ids>>filter(_.organization_type.isnull()))} without Organization Types")

Althought we still have 499 without Organization Types


In [77]:
ids>>filter(_.name.str.contains('Council'))>>select(_.name, _.organization_type)

,name,organization_type
61,Calaveras Council of Governments,Council of Governments
73,Cambria Community Council,Non-Profit Organization
384,Council of San Benito County Governments,NaN
436,Fresno Council of Governments,Council of Governments
499,Kern Council of Governments,Council of Governments
505,Lake County/City Area Planning Council,NaN
539,Mendocino Council of Governments,Council of Governments
650,Sacramento Area Council of Governments,Council of Governments
673,San Joaquin Council of Governments,Council of Governments
679,San Luis Obispo Council of Governments,Council of Governments


In [78]:
#all
ids

,name,ntd_id,itp_id,ct_code,district,organization_type,_merge
0,A-Paratransit,NaN,NaN,NaN,NaN,NaN,left_only
1,ABC Shuttle,NaN,NaN,NaN,NaN,Company,left_only
2,Able Inc.,NaN,NaN,NaN,NaN,Non-Profit Organization,left_only
3,Abrazar Inc.,NaN,NaN,NaN,NaN,Non-Profit Organization,left_only
4,Access Services,90157,1.0,6312.0,7.0,Independent Agency,both
...,...,...,...,...,...,...,...
1759,Leave Blank,NaN,NaN,5465.0,53.0,NaN,right_only
1760,U.S. Fish and Wildlife Service,NaN,NaN,6250.0,53.0,NaN,right_only
1761,Tidewater Southern Railway Company,NaN,NaN,6031.0,53.0,NaN,right_only
1762,Tri-Counties Regional Park Group,NaN,NaN,6176.0,53.0,NaN,right_only
